# Protótipo Know Your Fan (KYF) - E-sports

## Objetivo

Este notebook é um protótipo funcional que demonstra o conceito de uma solução "Know Your Fan" (KYF) focada em fãs de e-sports. O objetivo é mostrar o fluxo de coleta de dados, armazenamento e a *simulação conceitual* de processos de validação que poderiam ser usados para entender melhor e engajar os fãs, inspirado por estratégias de organizações como a FURIA.

## Tecnologias Utilizadas

* **Linguagem:** Python 3
* **Ambiente:** Google Colaboratory
* **Bibliotecas Principais:**
    * `sqlite3`: Para o banco de dados local simples.
    * `json`: Para armazenar dados semi-estruturados (listas, dicionários) em campos de texto.
    * `pandas`: Para exibição organizada de dados em tabelas.
    * `IPython.display`: Para renderizar Markdown e limpar output.
    * `re`: Para validações básicas (ex: formato CPF).

## Como Executar

1.  **Execute as Células de Setup (1, 2, 3):** Rode as três primeiras células de código em sequência para importar bibliotecas, inicializar o banco de dados (`kyf_database.db`) e definir as funções auxiliares.
    * **Importante:** Se você reiniciar o ambiente de execução do Colab, precisará rodar essas 3 células novamente.
2.  **Adicionar Fãs (Célula 5):** Execute a célula "📝 Coleta de Dados do Novo Fã". Siga as instruções interativas (inputs) para adicionar um ou mais fãs. Lembre-se da sensibilidade do CPF (use dados de teste).
3.  **Visualizar Resumo (Célula 6):** Execute a célula "📊 Visualizar Fãs Cadastrados" para ver uma lista de todos os fãs no banco.
4.  **Executar Simulações (Células 7 e 8):** Execute as células "🔗 Vínculo com Redes Sociais" e "🎮 Vínculo com Perfis de E-sports". Elas usarão um `target_fan_id` (atualmente ID 2) para buscar dados, explicar conceitos, mostrar resultados simulados e atualizar o status simulado no banco para aquele fã específico.

## Estrutura do Notebook

* **Células 1-3:** Configuração Inicial (Imports, DB Init, Funções Auxiliares).
* **Célula 4:** Título e Introdução Breve (Pode ser removida ou mesclada com esta).
* **Célula 5:** Coleta Interativa de Dados de um Novo Fã.
* **Célula 6:** Visualização de Resumo de Todos os Fãs.
* **Célula 7:** Simulação e Conceito - Vínculo Social.
* **Célula 8:** Simulação e Conceito - Vínculo Perfis E-sports.

## Funcional vs. Simulado

* **Funcional:**
    * Coleta de dados via input.
    * Processamento básico (mascaramento de CPF, formatação JSON).
    * Armazenamento e recuperação de dados no banco SQLite.
    * Visualização de dados via Pandas.
* **Simulado / Conceitual:**
    * **Vínculo Social:** A conexão real via OAuth e leitura de APIs NÃO é feita. A célula *descreve* como seria, exibe os links fornecidos e gera um *texto simulado* do resultado da validação, atualizando o campo `social_validation_sim` no banco.
    * **Vínculo Perfis E-sports:** A análise real via API/Scraping NÃO é feita. A célula *descreve* como seria, exibe os links fornecidos e gera um *texto simulado* do resultado da validação, atualizando o campo `esports_validation_sim` no banco.

## Privacidade (LGPD)

Este protótipo lida com dados fictícios ou de teste. Em uma aplicação real:
* O CPF só deve ser coletado com base legal clara e armazenado de forma segura (criptografado). A exibição deve ser sempre mascarada.
* O consentimento do usuário é essencial para coleta e processamento, especialmente para acesso a dados de terceiros (APIs sociais).
* Princípios de minimização de dados e transparência devem ser seguidos.

In [ ]:
# Célula 1: Importações Essenciais

# --- Bibliotecas Padrão do Python ---
import sqlite3  # Para interagir com o banco de dados SQLite (criar, inserir, consultar)
import json     # Para converter listas/dicionários Python em string formato JSON e vice-versa (útil para armazenar dados estruturados em campos de texto no DB)
import re       # Para usar Expressões Regulares (Regular Expressions), usadas aqui na validação básica do formato do CPF

import pandas as pd # Biblioteca poderosa para manipulação e análise de dados; usada aqui principalmente para exibir os dados do banco em tabelas formatadas (DataFrames)

# --- Módulos Específicos do Ambiente Jupyter/IPython ---
from IPython.display import display, Markdown, clear_output
# display: Função para renderizar objetos de forma rica no output (como tabelas pandas e Markdown)
# Markdown: Permite escrever e exibir texto formatado com Markdown diretamente via código Python
# clear_output: Função para limpar o output de uma célula programaticamente (útil em interfaces interativas)

# --- Mensagem de Confirmação ---
# Imprime uma mensagem simples para indicar que a célula foi executada e as bibliotecas (teoricamente) carregadas com sucesso.
print("Bibliotecas importadas.")


Bibliotecas importadas.


## Passo 1a: Configuração do Banco de Dados

A célula de código a seguir define o nome do nosso banco de dados SQLite e contém a função `init_db` que cria a tabela `fans` (se ela não existir). A função é executada ao final da célula para garantir que o banco esteja pronto.

In [ ]:
# Célula 2: Configuração e Inicialização do Banco de Dados SQLite

# --- Configuração do Banco de Dados ---
# Define o nome do arquivo que será usado para o banco de dados SQLite.
# Este arquivo será criado no diretório atual do ambiente Colab se não existir.
DB_NAME = 'kyf_database.db'

def init_db():
    """
    Conecta ao banco de dados SQLite e cria a tabela 'fans' se ela ainda não existir.
    Garante que a estrutura básica do banco esteja pronta para uso.
    É seguro executar esta função múltiplas vezes (devido ao 'IF NOT EXISTS').
    """
    try:
        # Conecta ao arquivo do banco de dados (cria o arquivo se não existir)
        conn = sqlite3.connect(DB_NAME)
        # Cria um cursor, que é usado para executar comandos SQL
        cursor = conn.cursor()

        # Comando SQL para criar a tabela 'fans' apenas se ela não existir ainda
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS fans (
                -- Colunas principais de identificação e dados básicos
                id INTEGER PRIMARY KEY AUTOINCREMENT, -- Chave primária única para cada fã, gerada automaticamente
                full_name TEXT NOT NULL,             -- Nome completo, obrigatório (NOT NULL)
                address TEXT,                        -- Endereço, opcional
                cpf_masked TEXT,                     -- CPF armazenado já mascarado (por segurança/LGPD)

                -- Colunas para interesses e atividades (armazenados como JSON strings)
                games_interest TEXT, -- Lista de jogos de interesse, ex: '["CS", "LoL"]'
                activities TEXT,     -- Lista de atividades, ex: '["Streams", "Casual"]'
                events_attended TEXT,-- Descrição textual livre
                esports_purchases TEXT,-- Descrição textual livre

                -- Colunas para status de validação (com valores padrão)
                id_doc_status TEXT DEFAULT 'Não verificado', -- Status da validação de ID (REMOVIDA do fluxo, mas mantida no schema por simplicidade)
                social_links TEXT DEFAULT '{}',             -- Dicionário de links sociais, ex: '{"twitter": "url", "twitch": "url"}'
                social_validation_sim TEXT DEFAULT 'Pendente', -- Guarda o RESULTADO SIMULADO da validação social
                esports_profiles TEXT DEFAULT '{}',         -- Dicionário de perfis e-sports, ex: '{"other": "url"}' (Steam foi removido)
                esports_validation_sim TEXT DEFAULT 'Pendente' -- Guarda o RESULTADO SIMULADO da validação de perfis e-sports
            )
        ''')
        # Confirma (salva) as alterações feitas no banco de dados (neste caso, a criação da tabela)
        conn.commit()
        # Fecha a conexão com o banco de dados
        conn.close()
        # Imprime mensagem de sucesso
        print(f"Banco de dados '{DB_NAME}' inicializado com sucesso. Tabela 'fans' pronta.")

    except Exception as e:
        # Captura e informa erros que possam ocorrer durante a inicialização do DB
        print(f"Erro ao inicializar o banco de dados: {e}")

# --- Inicializa o banco de dados ao executar esta célula ---
# Chama a função init_db() imediatamente para garantir que a tabela exista
# antes que outras células tentem interagir com ela.
init_db()


Banco de dados 'kyf_database.db' inicializado com sucesso. Tabela 'fans' pronta.


## Passo 1b: Definição das Funções Auxiliares

Esta célula define todas as funções Python que usaremos ao longo do notebook para tarefas repetitivas como mascarar CPF, adicionar/buscar/atualizar/deletar fãs no banco de dados. Executar esta célula torna as funções disponíveis para as células seguintes.

In [ ]:
# Célula 3: Funções Auxiliares para Manipulação de Dados (com Error Handling Refinado)

import sqlite3
import json
import re
import pandas as pd
from IPython.display import display, Markdown

DB_NAME = 'kyf_database.db'

def mask_cpf(cpf):
    """Mascara um CPF válido (11 dígitos numéricos) no formato ***.XXX.XXX-**."""
    if cpf and len(cpf) == 11 and cpf.isdigit():
        if re.fullmatch(r'\d{11}', cpf):
             return f"***.{cpf[3:6]}.{cpf[6:9]}-**"
    return "Inválido ou não fornecido"

def add_fan(data):
    """Adiciona um novo fã ao banco de dados, tratando campos ausentes."""
    conn = None # Inicializa conn fora do try para garantir que exista no finally (se usássemos)
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO fans (full_name, address, cpf_masked, games_interest, activities,
                              events_attended, esports_purchases, social_links, esports_profiles)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (data.get('full_name', 'Nome não fornecido'),
              data.get('address', ''), data.get('cpf_masked', 'Não fornecido'),
              data.get('games_interest', '[]'), data.get('activities', '[]'),
              data.get('events_attended', ''), data.get('esports_purchases', ''),
              data.get('social_links', '{}'), data.get('esports_profiles', '{}')
             ))
        fan_id = cursor.lastrowid
        conn.commit()
        print(f"Fã '{data.get('full_name')}' adicionado com ID: {fan_id}")
        return fan_id
    except sqlite3.Error as db_err: # Captura erros específicos do SQLite
        print(f"Erro de banco de dados ao adicionar fã: {db_err}")
        return None
    except Exception as e: # Captura outros erros inesperados
        print(f"Erro inesperado ao adicionar fã: {e}")
        return None
    finally:
        # Garante que a conexão seja fechada mesmo se ocorrer um erro
        if conn:
            conn.close()

def get_fan(fan_id):
    """Busca os dados de um fã pelo ID e retorna como dicionário."""
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM fans WHERE id = ?", (fan_id,))
        fan_data_row = cursor.fetchone()
        # A conexão pode ser fechada aqui, antes de processar o resultado
        conn.close()
        conn = None # Indica que foi fechada
        return dict(fan_data_row) if fan_data_row else None
    except sqlite3.Error as db_err:
        print(f"Erro de banco de dados ao buscar fã {fan_id}: {db_err}")
        return None
    except Exception as e:
        print(f"Erro inesperado ao buscar fã {fan_id}: {e}")
        return None
    finally:
        if conn: # Fecha caso ainda esteja aberta por algum erro antes do close() explícito
            conn.close()

def get_all_fans():
    """Busca um resumo de todos os fãs (ID, Nome, CPF Masked) para listagem."""
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute("SELECT id, full_name, cpf_masked FROM fans ORDER BY id")
        fans_rows = cursor.fetchall()
        conn.close()
        conn = None
        return [dict(fan) for fan in fans_rows]
    except sqlite3.Error as db_err:
        print(f"Erro de banco de dados ao buscar todos os fãs: {db_err}")
        return []
    except Exception as e:
        print(f"Erro inesperado ao buscar todos os fãs: {e}")
        return []
    finally:
        if conn:
            conn.close()

def update_fan_status(fan_id, field_name, status_value):
    """Atualiza um campo de status/simulação específico para um fã."""
    allowed_fields = ['id_doc_status', 'social_validation_sim', 'esports_validation_sim']
    if field_name not in allowed_fields:
        print(f"Erro: Campo '{field_name}' não permitido para atualização de status.")
        return False
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        query = f"UPDATE fans SET {field_name} = ? WHERE id = ?"
        cursor.execute(query, (status_value, fan_id))
        conn.commit()
        rows_affected = cursor.rowcount
        conn.close()
        conn = None
        if rows_affected > 0:
            print(f"Status '{field_name}' atualizado para '{status_value}' para o fã ID {fan_id}.")
            return True
        else:
            print(f"Fã com ID {fan_id} não encontrado para atualização de status '{field_name}'.")
            return False
    except sqlite3.Error as db_err:
        print(f"Erro de banco de dados ao atualizar status '{field_name}' do fã {fan_id}: {db_err}")
        return False
    except Exception as e:
        print(f"Erro inesperado ao atualizar status '{field_name}' do fã {fan_id}: {e}")
        return False
    finally:
        if conn:
            conn.close()

# --- Função para Deletar Fã ---
def delete_fan(fan_id):
    """Deleta um fã do banco de dados pelo seu ID."""
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        # Executa o comando DELETE para o ID fornecido. Usa placeholder '?' para segurança.
        cursor.execute("DELETE FROM fans WHERE id = ?", (fan_id,))
        conn.commit()
        rows_affected = cursor.rowcount # Verifica quantas linhas foram deletadas

        # Fecha a conexão ANTES de retornar ou imprimir sucesso/falha
        conn.close()
        conn = None

        if rows_affected > 0:
            # Se uma linha foi afetada, o fã foi deletado com sucesso.
            print(f"Fã com ID {fan_id} deletado com sucesso.")
            return True
        else:
            # Se nenhuma linha foi afetada, o fã com aquele ID não existia.
            print(f"Fã com ID {fan_id} não encontrado para deleção.")
            return False
    except sqlite3.Error as db_err:
        print(f"Erro de banco de dados ao deletar fã {fan_id}: {db_err}")
        return False
    except Exception as e:
        print(f"Erro inesperado ao deletar fã {fan_id}: {e}")
        return False
    finally:
        # Garante que a conexão seja fechada em caso de erro inesperado antes do close() explícito
        if conn:
            conn.close()

# Confirma que a célula foi processada (as definições foram lidas)
print("Funções auxiliares (mask_cpf, add_fan, get_fan, get_all_fans, update_fan_status, delete_fan) definidas (com error handling refinado).")

Funções auxiliares (mask_cpf, add_fan, get_fan, get_all_fans, update_fan_status, delete_fan) definidas (com error handling refinado).


## Passo 2: Adicionar um Novo Fã

A célula de código abaixo iniciará um processo interativo para coletar os dados de um novo fã. Responda às perguntas que aparecerão no output.

In [ ]:
# Célula 5: Coleta de Dados do Fã via Formulário Interativo (ipywidgets)

# --- Importações Essenciais ---
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import json
import re
from urllib.parse import urlparse
import pandas as pd # Para exibir no callback

# --- Redefinir Funções Auxiliares (ou garantir Célula 3) ---
def is_valid_url(url): # Exemplo
    if not url: return True
    try: result = urlparse(url); return all([result.scheme, result.netloc])
    except ValueError: return False
# !! Garanta que mask_cpf, add_fan, get_fan da Célula 3 estejam definidos !!

# --- Estilo Comum para Descrições ---
# Definir um estilo padrão para tentar alinhar a largura das descrições
# 'initial' tenta ajustar ao conteúdo. '180px' (ou outro valor) fixa a largura.
# Experimente 'initial' primeiro, se não ficar bom, tente um valor fixo como '180px'.
style_desc_longa = {'description_width': '180px'} # Ou 'initial'

# --- Criação dos Widgets do Formulário (com estilo e layout ajustados) ---

# Título Principal
title_label = widgets.HTML("<h2>📝 Coleta de Dados do Novo Fã</h2><p>Preencha as informações abaixo e clique em Salvar.</p><hr>")

# Seção: Dados Básicos
w_full_name = widgets.Text(description="Nome Completo:", placeholder="Digite o nome completo", style=style_desc_longa, layout={'width': '80%'})
w_address = widgets.Textarea(description="Endereço:", placeholder="Digite o endereço (opcional)", style=style_desc_longa, layout={'width': '80%', 'height': '60px'})
basic_info_section = widgets.VBox([widgets.HTML("<h3>Dados Básicos</h3>"), w_full_name, w_address], layout={'margin': '0 0 10px 0'}) # Margem inferior

# Seção: CPF
w_cpf = widgets.Text(description="CPF:", placeholder="Digite 11 números", style=style_desc_longa, layout={'width': '315px', 'margin': '0 0 5px 0'})
cpf_warning_label = widgets.HTML("<p><small><font color='red'><b>Atenção (LGPD):</b> Use apenas CPFs de teste (11 dígitos válidos).</font></small></p>")
cpf_section = widgets.VBox([widgets.HTML("<h3>CPF (Dado Sensível)</h3>"), w_cpf, cpf_warning_label], layout={'margin': '0 0 10px 0'}) # Margem inferior

# Seção: Interesses e Atividades
w_games = widgets.Text(description="Jogos (sep. vírgula):", placeholder="Ex: CS, Valorant, LoL", style=style_desc_longa, layout={'width': '80%'})
w_activities = widgets.Text(description="Atividades (sep. vírgula):", placeholder="Ex: Competitivo, Streams", style=style_desc_longa, layout={'width': '80%'})
w_events = widgets.Textarea(description="Eventos Comparecidos:", placeholder="Eventos online/presenciais comparecidos", style=style_desc_longa, layout={'width': '80%', 'height': '60px'})
w_purchases = widgets.Textarea(description="Compras E-sports:", placeholder="Produtos, Skins, Periféricos", style=style_desc_longa, layout={'width': '80%', 'height': '60px'})
interests_section = widgets.VBox([widgets.HTML("<h3>Interesses e Atividades em E-sports</h3>"), w_games, w_activities, w_events, w_purchases], layout={'margin': '0 0 10px 0'}) # Margem inferior

# Seção: Links
w_link_twitter = widgets.Text(description="Twitter URL:", placeholder="https://twitter.com/usuario", style=style_desc_longa, layout={'width': '80%'})
w_link_twitch = widgets.Text(description="Twitch URL:", placeholder="https://twitch.tv/canal", style=style_desc_longa, layout={'width': '80%'})
w_link_other_esports = widgets.Text(description="Outro Perfil URL:", placeholder="Link GamersClub, HLTV, etc.", style=style_desc_longa, layout={'width': '80%'})
links_section = widgets.VBox([widgets.HTML("<h3>Links de Perfis Online</h3>"), w_link_twitter, w_link_twitch, w_link_other_esports], layout={'margin': '0 0 10px 0'}) # Margem inferior

# --- Botão de Ação e Área de Output ---
w_button_save = widgets.Button(description="Salvar Fã", button_style='success', icon='save', layout={'margin': '10px 0 0 0'}) # Margem superior
output_area = widgets.Output()

# --- Função Callback (Exatamente a mesma de antes) ---
def on_save_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        print("Processando...")
        # 1. Ler valores
        name = w_full_name.value; address = w_address.value; cpf_raw = w_cpf.value
        games_str = w_games.value; activities_str = w_activities.value
        events = w_events.value; purchases = w_purchases.value
        link_twitter = w_link_twitter.value; link_twitch = w_link_twitch.value; link_other = w_link_other_esports.value
        # 2. Validar
        errors = []
        if not name: errors.append("Nome Completo é obrigatório.")
        if not (cpf_raw and len(cpf_raw) == 11 and cpf_raw.isdigit()):
            errors.append("CPF inválido. Deve conter 11 números.")
            cpf_masked = "Inválido ou não fornecido"
        else:
            try: cpf_masked = mask_cpf(cpf_raw) # Requer Célula 3
            except NameError: print("Erro: Função mask_cpf não definida."); return
        if not is_valid_url(link_twitter): print(">>> Aviso: Formato URL Twitter inválido.")
        if not is_valid_url(link_twitch): print(">>> Aviso: Formato URL Twitch inválido.")
        if not is_valid_url(link_other): print(">>> Aviso: Formato URL Outro Perfil inválido.")
        if errors:
            print("### Erros de Validação ###"); [print(f"- {e}") for e in errors]; return
        # 3. Processar
        try:
            games_list = [g.strip() for g in games_str.split(',') if g.strip()]; games_interest_json = json.dumps(games_list)
            activities_list = [a.strip() for a in activities_str.split(',') if a.strip()]; activities_json = json.dumps(activities_list)
            social_links_dict = {'twitter': link_twitter, 'twitch': link_twitch}; social_links_json = json.dumps(social_links_dict)
            esports_profiles_dict = {'other': link_other}; esports_profiles_json = json.dumps(esports_profiles_dict)
        except Exception as e: print(f"Erro ao processar listas/JSON: {e}"); return
        # 4. Montar e Salvar
        fan_data_to_save = {
            'full_name': name, 'address': address, 'cpf_masked': cpf_masked,
            'games_interest': games_interest_json, 'activities': activities_json,
            'events_attended': events, 'esports_purchases': purchases,
            'social_links': social_links_json, 'esports_profiles': esports_profiles_json
        }
        print("Salvando no banco de dados...")
        try:
            new_fan_id = add_fan(fan_data_to_save) # Requer Célula 3
            if new_fan_id: print(f"--- SUCESSO! Fã salvo com ID: {new_fan_id} ---")
            else: print("### Falha ao salvar o fã no banco de dados. Verifique os logs. ###")
        except NameError: print("Erro: Função add_fan não definida. Rode a Célula 3.")
        except Exception as e: print(f"Erro inesperado ao tentar salvar: {e}")

# --- Ligar a Função ao Botão ---
w_button_save.on_click(on_save_button_clicked)

# --- Exibir o Formulário e a Área de Output ---
form_layout = widgets.VBox([
    title_label,
    basic_info_section,
    cpf_section,
    interests_section,
    links_section,
    w_button_save
])
display(form_layout, output_area)

Output()

## Passo 3: Visualizar Resumo dos Fãs

A próxima célula buscará um resumo de todos os fãs atualmente no banco de dados e exibirá em uma tabela.

In [ ]:
# Célula 6: Visualizar Todos os Fãs Cadastrados (Resumo)

# --- Formatação da Interface no Output ---
display(Markdown("---"))
display(Markdown("## 📊 Visualizar Fãs Cadastrados"))

# --- Busca dos Dados ---
# Chama a função auxiliar (definida na Célula 3) que busca um resumo de todos os fãs.
# Espera-se que retorne uma lista de dicionários, cada um com 'id', 'full_name', 'cpf_masked'.
all_fans_summary = get_all_fans()

# --- Verificação e Exibição ---
# Verifica se a lista retornada está vazia (nenhum fã no banco)
if not all_fans_summary:
    # Imprime uma mensagem informativa se não houver fãs
    print("Nenhum fã cadastrado ainda.")
else:
    # Se a lista não estiver vazia, imprime quantos fãs foram encontrados
    print(f"Encontrados {len(all_fans_summary)} fãs no banco de dados:")

    # Cria um DataFrame do Pandas a partir da lista de dicionários.
    # O Pandas organiza os dados automaticamente em formato de tabela,
    # usando as chaves dos dicionários como nomes das colunas.
    fans_df = pd.DataFrame(all_fans_summary)

    # Usa a função display() (do IPython) para renderizar o DataFrame
    # como uma tabela HTML formatada no output da célula.
    # É mais visualmente agradável que um simples print(fans_df).
    display(fans_df)

# --- Formatação Final ---
display(Markdown("---"))

---

## 📊 Visualizar Fãs Cadastrados

Encontrados 4 fãs no banco de dados:


,id,full_name,cpf_masked
0,2,danyel,***.422.327-**
1,3,Beleti,***.260.108-**
2,4,Matheus,***.936.328-**
3,5,Pedro,***.585.638-**


---

## Passo 4: Simulação - Vínculo Social

Agora, vamos simular a validação dos links sociais. A célula seguinte pedirá um ID de fã, buscará seus links, explicará o conceito de validação real e salvará um resultado simulado no banco.

In [ ]:
# Célula 7: Vínculo com Redes Sociais (Conceitual / Simulado) - DINÂMICO

# Importar novamente para garantir (opcional se Célula 1 rodou)
import json
import pandas as pd
from IPython.display import display, Markdown

display(Markdown("---"))
display(Markdown("## 🔗 Vínculo com Redes Sociais (Conceitual / Simulado)"))

# --- Obter ID do Fã para Simulação --- ## MODIFICADO ##
# Pede ao usuário para digitar o ID do fã alvo
fan_id_input_sim = input("Digite o ID do fã para executar a simulação social: ")
target_fan_id = None # Inicializa como None para controle de erro
try:
    # Tenta converter o input para inteiro
    target_fan_id = int(fan_id_input_sim)
    # Imprime confirmação do ID que será usado
    print(f"Tentando executar simulação social para o Fã ID: {target_fan_id}")
except ValueError:
    # Informa erro se o input não for um número válido
    print("ID inválido. Por favor, digite um número.")
    # target_fan_id permanece None, e a lógica principal abaixo não será executada

# --- Executa a lógica principal apenas se um ID válido foi fornecido --- ## ADICIONADO IF AQUI ##
if target_fan_id is not None:
    # Busca os dados completos do fã alvo usando a função auxiliar
    fan_data = get_fan(target_fan_id)

    # Verifica se o fã com o ID especificado foi encontrado no banco
    if not fan_data:
        print(f"Fã com ID {target_fan_id} não encontrado.")
    else:
        # --- Bloco de Lógica da Simulação (sem alterações internas) ---

        # Se o fã foi encontrado, exibe para quem a simulação se aplica
        display(Markdown(f"**Simulando para o Fã:** {fan_data['full_name']} (ID: {target_fan_id})"))

        # --- Etapa 1: Exibir Links Sociais Fornecidos pelo Fã ---
        display(Markdown("\n**Links Sociais Fornecidos:**"))
        social_links_dict = {}
        try:
            loaded_links = json.loads(fan_data.get('social_links', '{}'))
            if isinstance(loaded_links, dict):
                social_links_dict = loaded_links
            if not social_links_dict or all(not v for v in social_links_dict.values()):
                 print("Nenhum link social fornecido ou encontrado para este fã.")
            else:
                 for platform, link in social_links_dict.items():
                     if link and isinstance(link, str) and link.strip():
                         display(Markdown(f"- {platform.capitalize()}: [{link}]({link})"))
                     else:
                         display(Markdown(f"- {platform.capitalize()}: Não fornecido"))
        except json.JSONDecodeError:
            print("Erro ao ler/decodificar links sociais do banco de dados.")
        except Exception as e:
            print(f"Erro inesperado ao processar links sociais: {e}")

        # --- Etapa 2: Explicação Conceitual (OAuth + API) ---
        display(Markdown("\n**Conceito de Validação/Leitura:**"))
        display(Markdown(
            "1.  **Autorização (OAuth 2.0):** ...\n"
            "2.  **Coleta via API:** ...\n"
            "    * **Twitter:** ...\n"
            "    * **Twitch:** ..."
        )) # Mantido abreviado para clareza da resposta

        # --- Etapa 3: Geração do Resultado Simulado ---
        display(Markdown("\n**Resultado da Validação (Simulado):**"))
        simulated_social_result = f"Simulação: Engajamento detectado no Twitter com posts sobre 'FURIA'. Segue os canais de 'Gaules' e 'Baiano' na Twitch."
        print(simulated_social_result)

        # --- Etapa 4: Atualização do Status Simulado no Banco de Dados ---
        update_fan_status(target_fan_id, 'social_validation_sim', simulated_social_result)

        # --- Etapa 5: Exibição do Status Atualizado para Confirmação ---
        display(Markdown("\n**Status Atualizado no Banco:**"))
        updated_fan_data = get_fan(target_fan_id)
        if updated_fan_data:
            display(pd.DataFrame([{'id': updated_fan_data['id'],
                                  'full_name': updated_fan_data['full_name'],
                                  'social_validation_sim': updated_fan_data['social_validation_sim']}]))
        else:
            print(f"Não foi possível verificar o status atualizado para o fã ID {target_fan_id}.")

# --- Formatação Final ---
display(Markdown("---"))


---

## 🔗 Vínculo com Redes Sociais (Conceitual / Simulado)

Tentando executar simulação social para o Fã ID: 2


**Simulando para o Fã:** danyel (ID: 2)


**Links Sociais Fornecidos:**

...


**Resultado da Validação (Simulado):**

Simulação: Engajamento detectado no Twitter com posts sobre 'FURIA'. Segue os canais de 'Gaules' e 'Baiano' na Twitch.
Status 'social_validation_sim' atualizado para 'Simulação: Engajamento detectado no Twitter com posts sobre 'FURIA'. Segue os canais de 'Gaules' e 'Baiano' na Twitch.' para o fã ID 2.



**Status Atualizado no Banco:**

,id,full_name,social_validation_sim
0,2,danyel,Simulação: Engajamento detectado no Twitter co...


---

## Passo 5: Simulação - Vínculo E-sports

Similarmente à etapa anterior, a próxima célula simulará a validação dos perfis de e-sports para um ID de fã especificado, explicando o conceito e salvando o resultado simulado.

In [ ]:
# Célula 8: Vínculo com Perfis de E-sports (Conceitual / Simulado) - DINÂMICO

# Importar novamente para garantir (opcional se Célula 1 rodou)
import json
import pandas as pd
from IPython.display import display, Markdown

display(Markdown("---"))
display(Markdown("## 🎮 Vínculo com Perfis de E-sports (Conceitual / Simulado)"))

# --- Obter ID do Fã para Simulação --- ## MODIFICADO ##
# Pede ao usuário para digitar o ID do fã alvo
fan_id_input_sim_esports = input("Digite o ID do fã para executar a simulação de e-sports: ")
target_fan_id = None # Inicializa como None para controle de erro
try:
    # Tenta converter o input para inteiro
    target_fan_id = int(fan_id_input_sim_esports)
    # Imprime confirmação do ID que será usado
    print(f"Tentando executar simulação de e-sports para o Fã ID: {target_fan_id}")
except ValueError:
    # Informa erro se o input não for um número válido
    print("ID inválido. Por favor, digite um número.")
    # target_fan_id permanece None, e a lógica principal abaixo não será executada

# --- Executa a lógica principal apenas se um ID válido foi fornecido --- ## ADICIONADO IF AQUI ##
if target_fan_id is not None:
    # Busca os dados completos do fã alvo.
    fan_data = get_fan(target_fan_id)

    # Verifica se o fã foi encontrado.
    if not fan_data:
        print(f"Fã com ID {target_fan_id} não encontrado.")
    else:
        # --- Bloco de Lógica da Simulação (sem alterações internas) ---

        # Se encontrou, exibe para quem a simulação se aplica.
        display(Markdown(f"**Simulando para o Fã:** {fan_data['full_name']} (ID: {target_fan_id})"))

        # --- Etapa 1: Exibir Links de Perfis de E-sports Fornecidos ---
        display(Markdown("\n**Perfis de E-sports Fornecidos:**"))
        esports_profiles_dict = {}
        try:
            loaded_profiles = json.loads(fan_data.get('esports_profiles', '{}'))
            if isinstance(loaded_profiles, dict):
                esports_profiles_dict = loaded_profiles
            if not esports_profiles_dict or all(not v for v in esports_profiles_dict.values()):
                 print("Nenhum perfil de e-sports fornecido ou encontrado para este fã.")
            else:
                 for platform, link in esports_profiles_dict.items():
                     if link and isinstance(link, str) and link.strip():
                         display(Markdown(f"- {platform.capitalize()}: [{link}]({link})"))
        except json.JSONDecodeError:
            print("Erro ao ler/decodificar perfis de e-sports do banco de dados.")
        except Exception as e:
             print(f"Erro inesperado ao processar perfis de e-sports: {e}")

        # --- Etapa 2: Explicação Conceitual (API / Scraping) ---
        display(Markdown("\n**Conceito de Validação/Análise:**"))
        display(Markdown(
            "1.  **Coleta de Dados:** ...\n"
            "2.  **Extração de Métricas:** ...\n"
            "    * **Horas Jogadas:** ...\n"
            "    * **Rank/Patente/Nível:** ...\n"
            "    * **Histórico:** ...\n"
            "    * **Conexões:** ...\n"
            "3.  **Análise (IA Opcional):** ..."
        )) # Mantido abreviado para clareza da resposta

        # --- Etapa 3: Geração do Resultado Simulado ---
        display(Markdown("\n**Resultado da Validação (Simulado):**"))
        simulated_esports_result = f"Simulação: Perfil 'other' analisado. Detectado Rank 'Platina' em plataforma competitiva. Histórico de partidas consistente com interesse em 'Valorant'."
        print(simulated_esports_result)

        # --- Etapa 4: Atualização do Status Simulado no Banco de Dados ---
        update_fan_status(target_fan_id, 'esports_validation_sim', simulated_esports_result)

        # --- Etapa 5: Exibição do Status Atualizado para Confirmação ---
        display(Markdown("\n**Status Atualizado no Banco:**"))
        updated_fan_data = get_fan(target_fan_id)
        if updated_fan_data:
            display(pd.DataFrame([{'id': updated_fan_data['id'],
                                  'full_name': updated_fan_data['full_name'],
                                  'esports_validation_sim': updated_fan_data['esports_validation_sim']}]))
        else:
            print(f"Não foi possível verificar o status atualizado para o fã ID {target_fan_id}.")

# --- Formatação Final ---
display(Markdown("---"))


---

## 🎮 Vínculo com Perfis de E-sports (Conceitual / Simulado)

Tentando executar simulação de e-sports para o Fã ID: 3


**Simulando para o Fã:** Beleti (ID: 3)


**Perfis de E-sports Fornecidos:**

...


**Resultado da Validação (Simulado):**

Simulação: Perfil 'other' analisado. Detectado Rank 'Platina' em plataforma competitiva. Histórico de partidas consistente com interesse em 'Valorant'.
Status 'esports_validation_sim' atualizado para 'Simulação: Perfil 'other' analisado. Detectado Rank 'Platina' em plataforma competitiva. Histórico de partidas consistente com interesse em 'Valorant'.' para o fã ID 3.



**Status Atualizado no Banco:**

,id,full_name,esports_validation_sim
0,3,Beleti,Simulação: Perfil 'other' analisado. Detectado...


---

## Passo 6: Visualizar Perfil Completo

Esta célula permite visualizar todos os dados armazenados e simulados para um único fã. Ela pedirá o ID do fã desejado.

In [ ]:
# Célula 9: Visualizar Perfil Completo de um Fã

# Importar novamente caso o kernel tenha sido reiniciado
import json
import pandas as pd
from IPython.display import display, Markdown

display(Markdown("---")) # Separador antes da célula
display(Markdown("## 👤 Visualizar Perfil Completo do Fã"))

# --- Obter ID do Fã ---
fan_id_input = input("Digite o ID do fã que deseja visualizar: ")
try:
    fan_id_to_view = int(fan_id_input)
except ValueError:
    print("ID inválido. Por favor, digite um número.")
    fan_id_to_view = None # Indica erro

# --- Buscar e Exibir Dados ---
if fan_id_to_view is not None:
    # Busca os dados do fã (requer que get_fan da Célula 3 tenha sido executada)
    fan_profile_data = get_fan(fan_id_to_view)

    if not fan_profile_data:
        print(f"\nFã com ID {fan_id_to_view} não encontrado.")
    else:
        # Exibe o título do perfil
        display(Markdown(f"\n### Perfil de: **{fan_profile_data['full_name']}** (ID: {fan_profile_data['id']})"))

        # --- Dados Básicos ---
        display(Markdown("#### Dados Básicos"))
        display(Markdown(f"- **Endereço:** {fan_profile_data.get('address') or 'Não informado'}"))
        display(Markdown(f"- **CPF (Mascarado):** {fan_profile_data.get('cpf_masked') or 'Não informado'}"))

        # --- Separador Visual --- ## ADICIONADO ##
        display(Markdown("---"))

        # --- Interesses e Atividades ---
        display(Markdown("#### Interesses e Atividades"))
        try:
            # Processa e exibe lista de jogos como marcadores ## ALTERADO ##
            display(Markdown("- **Jogos de Interesse:**"))
            games_list = json.loads(fan_profile_data.get('games_interest', '[]'))
            if games_list:
                # Itera sobre a lista e exibe cada jogo como um item de marcador
                for game in games_list:
                    display(Markdown(f"  - {game}")) # Usa marcador "-"
            else:
                # Mensagem se a lista estiver vazia
                display(Markdown("  - Não informado"))

            # Processa e exibe lista de atividades como marcadores ## ALTERADO ##
            display(Markdown("- **Atividades:**"))
            activities_list = json.loads(fan_profile_data.get('activities', '[]'))
            if activities_list:
                # Itera sobre a lista e exibe cada atividade como um item de marcador
                for activity in activities_list:
                    display(Markdown(f"  - {activity}")) # Usa marcador "-"
            else:
                 # Mensagem se a lista estiver vazia
                display(Markdown("  - Não informado"))

        except json.JSONDecodeError:
            display(Markdown("- *Erro ao processar interesses/atividades (JSON inválido)*"))

        # Campos de texto livre (sem alteração na formatação)
        display(Markdown(f"- **Eventos Comparecidos:** {fan_profile_data.get('events_attended') or 'Não informado'}"))
        display(Markdown(f"- **Compras Relacionadas:** {fan_profile_data.get('esports_purchases') or 'Não informado'}"))

        # --- Separador Visual --- ## ADICIONADO ##
        display(Markdown("---"))

        # --- Links Online ---
        display(Markdown("#### Links e Perfis Online"))
        # Processa e exibe links sociais (formato anterior com marcadores já era bom)
        display(Markdown("**Links Sociais:**"))
        try:
            social_links_dict = json.loads(fan_profile_data.get('social_links', '{}'))
            if not social_links_dict or all(not v for v in social_links_dict.values()):
                display(Markdown("  - Nenhum link social fornecido."))
            else:
                for platform, link in social_links_dict.items():
                    if link and isinstance(link, str) and link.strip():
                         display(Markdown(f"  - {platform.capitalize()}: [{link}]({link})"))
                    else:
                         display(Markdown(f"  - {platform.capitalize()}: Não fornecido"))
        except json.JSONDecodeError:
             display(Markdown("- *Erro ao processar links sociais (JSON inválido)*"))

        # Processa e exibe links de e-sports (formato anterior com marcadores já era bom)
        display(Markdown("**Perfis E-sports:**"))
        try:
            esports_profiles_dict = json.loads(fan_profile_data.get('esports_profiles', '{}'))
            if not esports_profiles_dict or all(not v for v in esports_profiles_dict.values()):
                display(Markdown("  - Nenhum perfil de e-sports fornecido."))
            else:
                for platform, link in esports_profiles_dict.items():
                     if link and isinstance(link, str) and link.strip():
                         display(Markdown(f"  - {platform.capitalize()}: [{link}]({link})"))
        except json.JSONDecodeError:
             display(Markdown("- *Erro ao processar perfis e-sports (JSON inválido)*"))

        # --- Separador Visual --- ## ADICIONADO ##
        display(Markdown("---"))

        # --- Status das Validações Simuladas ---
        display(Markdown("#### Status das Validações (Simuladas)"))
        display(Markdown(f"- **Validação Social:** {fan_profile_data.get('social_validation_sim') or 'Pendente'}"))
        display(Markdown(f"- **Validação Perfis E-sports:** {fan_profile_data.get('esports_validation_sim') or 'Pendente'}"))

# --- Formatação Final ---
display(Markdown("---")) # Separador após a célula

## Passo 7: Deletar um Fã

Caso precise remover dados de teste, a célula a seguir pedirá o ID de um fã e tentará deletar o registro correspondente do banco de dados, mostrando a lista atualizada em seguida.

In [9]:
# Célula 10: Deletar um Fã

# Importar display/Markdown se for usar nesta célula
from IPython.display import display, Markdown
import pandas as pd # Para exibir a lista atualizada

display(Markdown("---"))
display(Markdown("## 🗑️ Deletar Fã"))

# --- Obter ID do Fã para Deletar ---
fan_id_input_delete = input("Digite o ID do fã que deseja DELETAR: ")
target_delete_id = None # Inicializa
try:
    target_delete_id = int(fan_id_input_delete)
except ValueError:
    print("ID inválido. Por favor, digite um número.")
    # target_delete_id permanece None

# --- Tentar Deletar (apenas se o ID for válido) ---
if target_delete_id is not None:
    # Chama a função auxiliar definida na Célula 3
    deleted = delete_fan(target_delete_id)
    # A função delete_fan já imprime mensagens de sucesso ou falha

    # --- Mostrar Lista Atualizada (Confirmação Visual) ---
    # Após tentar deletar, busca e exibe novamente a lista de fãs
    # para que possamos ver se o fã foi realmente removido.
    display(Markdown("\n**Lista de Fãs Atualizada:**"))
    all_fans_summary_after_delete = get_all_fans() # Requer que get_all_fans esteja definida (Célula 3)

    if not all_fans_summary_after_delete:
        print("Nenhum fã restou no banco de dados.")
    else:
        print(f"Atualmente há {len(all_fans_summary_after_delete)} fãs no banco:")
        fans_df_after_delete = pd.DataFrame(all_fans_summary_after_delete)
        display(fans_df_after_delete)
else:
    # Mensagem se o ID digitado inicialmente era inválido
    print("Nenhuma ação de deleção realizada devido a ID inválido.")


display(Markdown("---"))

---

## 🗑️ Deletar Fã

KeyboardInterrupt: Interrupted by user